In [11]:
from __future__ import division
%pylab inline
import numpy as np
import scipy.spatial.distance as dist
from scipy import stats
import pickle
import math
import collections
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [ ]:
def readRough(name="traininglabels"):
    with open(name) as label:
        labels = []
        for line in label:
            labels.append(line[0])
    return labels

In [ ]:
label = readRough()
print(len(label))
class_ = sorted(collections.Counter(label).items())
print(class_)

In [ ]:
with open('labelstats.txt', 'wb') as fp:
    pickle.dump(class_, fp)

In [ ]:
with open ('labelstats.txt', 'rb') as fp:
    class_ = pickle.load(fp)

In [ ]:
print class_

In [ ]:
def inputDigit(name="trainingimages"):
    with open(name) as digitTxt:
        image = [list(line)[0:28] for line in digitTxt]
        rows = len(image)
    return image


In [ ]:
data_training = inputDigit()

In [ ]:
# " " = high   =  1
# "%" = low    =  0
# len(row) = 10
def countNum(name = "yes_train.txt"):
    with open(name) as dataset:
        num_lines = sum(1 for line in dataset)
        return num_lines / 28
    


for training 

In [ ]:
# label yes = 1
# label no = 0
Num_sample_yes =  countNum()
Num_sample_no =  countNum("no_train.txt")
print(Num_sample_yes,Num_sample_no)

In [ ]:
Num_test_yes =  countNum("yes_test.txt")
Num_test_no =  countNum("no_test.txt")
print(Num_test_yes,Num_test_no)

In [ ]:
def inputDigit(name="yes_train.txt"):
    with open(name) as digitTxt:
        image = [list(line)[0:10] for line in digitTxt]
    return image

In [ ]:
data_yes = inputDigit()
data_no = inputDigit("no_train.txt")
test_yes = inputDigit("yes_test.txt")
test_no = inputDigit("no_test.txt")

In [ ]:
#   yes is class 0
#   no is class 1
k = 1
    
def training(yes_train, no_train, yes_num, no_num):
    # " " = high   =  1
    # "%" = low    =  0
    
    class_total = [yes_num, no_num]

    training0 = np.zeros(shape=(2, 25, 10))   #high
    training1 = np.zeros(shape=(2, 25, 10))   #low

    count = 0
    for i in range(int(yes_num)):
        for row in range(25):
            for col in range(10):
                if yes_train[i*28 + row][col] == ' ':
                    training1[0][row][col] += 1
                    training0[0][row][col] += 0
                else:
                    training1[0][row][col] += 0
                    training0[0][row][col] += 1
                    
    for i in range(int(no_num)):
        for row in range(25):
            for col in range(10):
                if no_train[i*28 + row][col] == ' ':
                    training1[1][row][col] += 1
                    training0[1][row][col] += 0
                else:
                    training1[1][row][col] += 0
                    training0[1][row][col] += 1

    for i in range(2):
        training1[i] = (training1[i] + k) / (class_total[i] + k * 2)
        training0[i] = (training0[i] + k) / (class_total[i] + k * 2)
#     print(training0[1][0])
#     print(training1[1][0])
    
    return training0, training1

In [ ]:
training0, training1 = training(data_yes, data_no, Num_sample_yes, Num_sample_no)

In [ ]:
# log P(class) + log P(f1,1 | class) + log P(f1,2 | class) + ... + log P(f28,28 | class)

# test_rough = inputDigit(name = "digitdata/testimages")
def testing(yes_test, no_test, Num_test_yes, Num_test_no):

    data_test = np.concatenate( (yes_test, no_test) , axis = 0 )
# #     print len(data_test) / 28
#     return 

    total = Num_test_yes + Num_test_no
    answer = np.zeros(int(total))
    
    class_total = [Num_test_yes, Num_test_no]
    
    for i in range(int(total)):
        test_image = np.zeros(shape=(25,10))
        for row in range(25):
            for col in range(10):
                if data_test[i*28+row][col] == ' ':
                    test_image[row][col] = 1
                else:
                    test_image[row][col] = 0
                    

        posteriori = np.zeros(2)
        for class_num in range(2):
            posteriori[class_num] = math.log(class_total[class_num])
            for row in range(25):
                for col in range(10):
                    if test_image[row][col] == 0:
                        posteriori[class_num] += math.log(training0[class_num][row][col])
                    else:
                        posteriori[class_num] += math.log(training1[class_num][row][col] )   
        answer[i] = np.argmax(posteriori)

    print(answer)



In [ ]:
testing(test_yes, test_no, Num_test_yes, Num_test_no)

================================================================
ec starts from here
================================================================

In [ ]:
# combination of 8 y/n
# sequence = label
# 1 = yes, 0 = no

# each dataset has shape 25 x 150


In [1]:
import glob
import numpy as np

In [2]:
def readFiles():
    files = glob.glob("training/*.txt")

    
    #threshold is the start of the slicing
    #set threshold to converge the best solution
    
    # 1 = yes, 0 = no
    
    training0 = []    # 25 * 10 for each sample and space 3 lines
    training1 = []  
    
    for fileName in files:
        with open(fileName) as file_:
            # fileName get label 
            labels = fileName[9:-4].split("_")
            # zero padding = [3, 10]
            
            padding = np.zeros(shape = (3, 10))
            
#             print len(labels)
            threshold = 1
            

            
            # slicing it and seperate into yes_train and no_train
            # " " = high   =  1
            # "%" = low    =  0
            rawdata = np.zeros(shape = (25,150))
            counter = 0 # count num of lines
            for line in file_:
                for i in range(len(line)-1):
                    if line[i] == " ":
                        rawdata[counter][i] = 1
                counter += 1
            
            
            newstart = 0
            num_slicing = 0
            
            for cow in range(len(line)-1):
                if num_slicing >= 8:
                    break
                if cow < newstart:
                    continue
                if np.sum(rawdata[:,cow]) > threshold:
                    #start slicing
#                     print num_slicing
                    if labels[num_slicing] == "0":   #no                        
                        if len(training0) == 0:
                            training0 = rawdata[:,cow:cow+10]
                        else:
                            training0 = np.concatenate( (training0,rawdata[:,cow:cow+10]),  axis = 0)
                        
                        training0 = np.concatenate( (training0,padding),  axis = 0)
                    else:
                        if len(training1) == 0:
                            training1 = rawdata[:,cow:cow+10]
                        else:
                            training1 = np.concatenate( (training1,rawdata[:,cow:cow+10]),  axis = 0)                              
                        training1 = np.concatenate( (training1,padding),  axis = 0)
                    
                    newstart = cow + 10
                    
                    #after slicing
                    num_slicing += 1
#     print len(training0)/28, len(training1)/28
    return (training1,training0)   # (yes,no)


In [3]:
trainingdata = readFiles()

yes_num = len(trainingdata[0])/28
no_num = len(trainingdata[1])/28

print yes_num, no_num

251 229


In [4]:
#   yes is class 0
#   no is class 1
k = 1
    
def training(yes_train, no_train, yes_num, no_num):
    # " " = high   =  1
    # "%" = low    =  0
    
    class_total = [yes_num, no_num]

    training0 = np.zeros(shape=(2, 25, 10))   #low
    training1 = np.zeros(shape=(2, 25, 10))   #high

    count = 0
    for i in range(int(yes_num)):
        for row in range(25):
            for col in range(10):
                if yes_train[i*28 + row][col] == 1:   #high
                    training1[0][row][col] += 1
                    training0[0][row][col] += 0
                else:
                    training1[0][row][col] += 0
                    training0[0][row][col] += 1
                    
    for i in range(int(no_num)):
        for row in range(25):
            for col in range(10):
                if no_train[i*28 + row][col] == 1:   #high
                    training1[1][row][col] += 1
                    training0[1][row][col] += 0
                else:
                    training1[1][row][col] += 0
                    training0[1][row][col] += 1

    for i in range(2):
        training1[i] = (training1[i] + k) / (class_total[i] + k * 2)
        training0[i] = (training0[i] + k) / (class_total[i] + k * 2)
#     print(training0[1][0])
#     print(training1[1][0])
    
    return training0, training1

In [5]:
training0, training1 = training(trainingdata[0],trainingdata[1],yes_num,no_num)

In [ ]:
# print trainingYes

In [6]:
def readTestFiles(directory):
    padding = np.zeros(shape = (3, 10))    
    files = glob.glob(directory)
    
    testingdata = []
    
    for fileName in files:
        with open(fileName) as file_:
            sample = [list(line)[0:10] for line in file_]
        if len(testingdata) == 0:
            testingdata = sample
        else:
            testingdata = np.concatenate((testingdata,sample), axis = 0)
        testingdata = np.concatenate((testingdata,padding), axis = 0)

    return testingdata
        

In [7]:
test_no = readTestFiles("no_test/*.txt")
test_yes = readTestFiles("yes_test/*.txt")

In [8]:
print len(test_no) / 28
print len(test_yes) / 28


50
50


In [9]:
# log P(class) + log P(f1,1 | class) + log P(f1,2 | class) + ... + log P(f28,28 | class)

# test_rough = inputDigit(name = "digitdata/testimages")
def testing(yes_test, no_test, Num_test_yes, Num_test_no):

    data_test = np.concatenate( (yes_test, no_test) , axis = 0 )
# #     print len(data_test) / 28
#     return 

    total = Num_test_yes + Num_test_no
    answer = np.zeros(int(total))
    
    class_total = [Num_test_yes, Num_test_no]
    
    for i in range(int(total)):
        test_image = np.zeros(shape=(25,10))
        for row in range(25):
            for col in range(10):
                if data_test[i*28+row][col] == ' ':
                    test_image[row][col] = 1
                else:
                    test_image[row][col] = 0
                    

        posteriori = np.zeros(2)
        for class_num in range(2):
            posteriori[class_num] = math.log(class_total[class_num])
            for row in range(25):
                for col in range(10):
                    if test_image[row][col] == 0:
                        posteriori[class_num] += math.log(training0[class_num][row][col])
                    else:
                        posteriori[class_num] += math.log(training1[class_num][row][col] )   
        answer[i] = np.argmax(posteriori)
    print answer
    return answer



In [12]:
result = testing(test_yes, test_no, 50, 50)

acc = 0

for i in range(len(result)):
    if i < 50:
        if result[i] == 0:
            acc += 1/100
    else:
        if result[i] == 1:
            acc += 1/100

print acc

[ 0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  1.
  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
0.96
